https://deepnote.com/app/leonard-puttmann-a8ef/Titanic-Dataset-544c6818-f79a-4068-bbc1-d6fdf42d2998

## Vorbereitung des Datensatzes für das Modell

Der Titanic-Datensatz wird für das Training eines maschinellen Lernmodells vorbereitet. Dabei wird der Datensatz in **X** (Merkmale) und **y** (Zielvariable) unterteilt.  

**X** enthält die wichtigsten Features, die entweder eine hohe Korrelation aufweisen oder als relevant erscheinen. Diese sind in `feature_cols` definiert.  

**y** enthält die Zielvariable `survived`, die angibt, ob eine Person überlebt hat (1) oder nicht (0). Da das Modell seine Vorhersagen anhand dieser Variable überprüft und anpasst, handelt es sich um **überwachtes maschinelles Lernen**.  


In [ ]:
import pandas as pd
import seaborn as sns

# Titanic-Datensatz aus Seaborn laden
titanic = sns.load_dataset("titanic").copy()

# Fehlende Werte auffüllen
titanic["age"] = titanic["age"].fillna(titanic["age"].median())
titanic["embarked"] = titanic["embarked"].fillna(titanic["embarked"].mode()[0])
titanic["fare"] = titanic["fare"].fillna(titanic["fare"].median())

# Feature-Auswahl
feature_cols = ["pclass", "age", "parch", "sex", "sibsp", "embarked"]

# Auswahl der Features für X
X = titanic.loc[:, feature_cols]
print("Shape von X:", X.shape)

# Zielvariable y (hat die Person überlebt?)
y = titanic["survived"]
print("Shape von y:", y.shape)

Shape von X: (891, 6)
Shape von y: (891,)


## Aufbau eines maschinellen Lernmodells  

Wir setzen ein Modell auf, das vorhersagen soll, ob eine Person auf der Titanic überlebt hat. Dazu nutzen wir zunächst eine **logistische Regression**, da sie einfach umzusetzen ist.  

Die logistische Regression analysiert die Abhängigkeit zwischen einer **binären Zielvariable** (0 = nicht überlebt, 1 = überlebt) und mehreren unabhängigen Variablen.  

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Logistisches Regressionsmodell erstellen
logistic_reg = LogisticRegression()

# Training mit X und y
logistic_reg.fit(X, y)

# Vorhersagen auf den Trainingsdaten treffen
pred_lr = logistic_reg.predict(X)

# Genauigkeit berechnen und in Prozent ausgeben
accuracy = accuracy_score(y, pred_lr) * 100
print(f"Genauigkeit: {accuracy:.2f}%")

Genauigkeit: 79.80%


##### Erste Genauigkeit des Modells  

Ohne Hyperparameter-Tuning erreicht unser Modell bereits eine Genauigkeit von ca. **80 %**.  
Dieser Wert kann je nach Datensplit leicht schwanken.  

Nun setzen wir eine **Pipeline** auf und nutzen **GridSearchCV**, um die Modellleistung weiter zu optimieren.  
Dadurch können wir die besten Hyperparameter automatisch ermitteln und die Genauigkeit möglicherweise verbessern.

## Aufteilung der Daten in Trainings- und Testsatz  

Bevor wir das Modell weiter optimieren, teilen wir die Daten in einen **Trainings-** und einen **Testsatz** auf.  
- **Trainingssatz**: Wird genutzt, um das Modell zu trainieren.  
- **Testsatz**: Bleibt unberührt, um später die Modellleistung auf unbekannten Daten zu überprüfen.  

Dies stellt sicher, dass unser Modell **nicht nur die Trainingsdaten auswendig lernt**, sondern auf neue Daten verallgemeinern kann.

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Aufteilen in Trainings- und Testdaten (Testdaten: 20%, Trainingsdaten: 80%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline mit StandardScaler und log. Regression
pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('logistic_Reg', LogisticRegression(max_iter=500, solver='liblinear'))  # ⚠️ Solver angepasst
])

# Hyperparameter für GridSearch
werte_raster = {
    'logistic_Reg__C': np.logspace(-4, 4, 5),
    'logistic_Reg__penalty': ['l1', 'l2']  # ⚠️ 'none' entfernt
}

# GridSearchCV mit Kreuzvalidierung
model_lr = GridSearchCV(pipe, param_grid=werte_raster, cv=5, verbose=True)

# Training des Modells
model_lr.fit(X_train, y_train)

# Beste Parameter ausgeben
print(f"Beste Parameter: {model_lr.best_params_}")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Beste Parameter: {'logistic_Reg__C': np.float64(0.01), 'logistic_Reg__penalty': 'l1'}
